In [1]:
import pandas as pd

existing_df = pd.read_csv('FinalProcessedDataset_02.csv')

new_df = pd.read_csv('sri_lanka_trending_analysis_15.csv')


In [2]:
new_df.columns

Index(['title', 'channel_name', 'video_id', 'channel_id', 'publish_date',
       'snapshot_date', 'country', 'view_count', 'like_count', 'comment_count',
       'description', 'thumbnail_url', 'video_tags', 'kind', 'language',
       'category_id', 'duration', 'definition', 'caption', 'privacy_status',
       'license', 'embeddable', 'live_broadcast_content', 'is_trending',
       'engagement_rate', 'collected_at', 'days_since_publish'],
      dtype='object')

In [5]:
new_df['snapshot_date'] = pd.to_datetime(new_df['snapshot_date'],utc=True)
#new_df['collected_at'] = pd.to_datetime(new_df['collected_at'],format="mixed")
new_df['publish_date'] = pd.to_datetime(new_df['publish_date'],utc=True)


new_df = new_df.drop_duplicates(subset=["video_id"], keep="last")


In [7]:
new_df.dropna(subset=['publish_date'], inplace=True)

In [8]:
from langdetect import detect, DetectorFactory
import numpy as np


# For consistent results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        if pd.isna(text) or text == '':
            return np.nan
        
        text_str = str(text)
        
        # Check if text contains Sinhala characters (Unicode range for Sinhala)
        has_sinhala = any('\u0D80' <= char <= '\u0DFF' for char in text_str)
        
        # If text contains Sinhala characters, return 'si' (Sinhala)
        if has_sinhala:
            return 'si'
        
        # Otherwise, use the original language detection
        return detect(text_str)
    except:
        return np.nan

# Apply language detection only to rows with null language
null_language_mask = new_df['language'].isna()
new_df.loc[null_language_mask, 'language'] = new_df.loc[null_language_mask, 'title'].apply(detect_language)

In [9]:
new_df[(new_df.language.isnull())]

,title,channel_name,video_id,channel_id,publish_date,snapshot_date,country,view_count,like_count,comment_count,...,definition,caption,privacy_status,license,embeddable,live_broadcast_content,is_trending,engagement_rate,collected_at,days_since_publish


In [10]:
# new_df['publish_date'] = pd.to_datetime(new_df['publish_date'],format='mixed')
new_df['snapshot_date'] = pd.to_datetime(new_df['snapshot_date'])
# Ensure publish_date exists
if "publish_date" in new_df.columns:
    new_df["publish_weekday"] = new_df["publish_date"].dt.weekday  # 0=Mon..6=Sun
    new_df["publish_hour"]    = new_df["publish_date"].dt.hour
else:
    raise ValueError("publish_date column is required for time EDA.")

new_df["is_weekend"] = new_df["publish_weekday"].isin([5,6]).astype(int)
new_df["slot_id"]    = new_df["publish_weekday"] * 24 + new_df["publish_hour"]

# Coarse parts of day (helpful for robust visual summaries)
bins = [0,6,12,18,24]
labels = ["Night(0-6)","Morning(6-12)","Afternoon(12-18)","Evening(18-24)"]
new_df["part_of_day"] = pd.cut(new_df["publish_hour"], bins=bins, labels=labels, right=True, include_lowest=True)


In [11]:

# Mapping dictionary: id → category name
category_map = {
    1: "Film & Animation",
    2: "Autos & Vehicles",
    10: "Music",
    15: "Pets & Animals",
    17: "Sports",
    18: "Short Movies",
    19: "Travel & Events",
    20: "Gaming",
    21: "Videoblogging",
    22: "People & Blogs",
    23: "Comedy",
    24: "Entertainment",
    25: "News & Politics",
    26: "Howto & Style",
    27: "Education",
    28: "Science & Technology",
    29: "Nonprofits & Activism",
    30: "Movies",
    31: "Anime/Animation",
    32: "Action/Adventure",
    33: "Classics",
    34: "Comedy",
    35: "Documentary",
    36: "Drama",
    37: "Family",
    38: "Foreign",
    39: "Horror",
    40: "Sci-Fi/Fantasy",
    41: "Thriller",
    42: "Shorts",
    43: "Shows",
    44: "Trailers",
    -1:"Unknown"
}
new_df['category_name'] = new_df['category_id'].map(category_map)

In [13]:
new_df['days_since_publish'] = new_df['days_since_publish'] + 1
new_df["views_per_hour"] = new_df["view_count"] / ((new_df['days_since_publish']+1)*24)


In [14]:
import holidays
sl_holidays = holidays.SriLanka(years=[2022, 2023, 2024, 2025])
holiday_dates = pd.to_datetime(list(sl_holidays.keys()))

new_df["is_holiday"] = new_df["publish_date"].dt.normalize().isin(holiday_dates)

In [15]:
# Convert publish_date to timezone-naive dates
new_df["publish_date_naive"] = new_df["publish_date"].dt.tz_localize(None).dt.normalize()

# Convert holiday dates also to timezone-naive
holiday_dates_naive = pd.to_datetime(list(sl_holidays.keys())).normalize()

# Check if publish date is a holiday
new_df["is_holiday"] = new_df["publish_date_naive"].isin(holiday_dates_naive)


In [16]:
new_df["title_len"] = new_df["title"].str.len()
new_df["desc_len"]  = new_df["description"].str.len()
new_df["tag_count"] = new_df["video_tags"].apply(lambda s: len([t for t in str(s).split(",") if t.strip()]))
new_df["has_hashtag"] = new_df["title"].str.contains("#").astype(int)

In [17]:
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

# Min-max normalize
scaler = MinMaxScaler()
new_df["views_per_hour_norm"] = scaler.fit_transform(new_df[["views_per_hour"]])


In [18]:
new_df['duration_timedelta'] = pd.to_timedelta(new_df['duration'])

# Convert the Timedelta to total seconds and then to minutes
new_df['duration_minutes'] = new_df['duration_timedelta'].dt.total_seconds() / 60


In [34]:
new_df.isnull().sum()

title                     0
channel_name              0
video_id                  0
channel_id                0
publish_date              0
snapshot_date             0
country                   0
view_count                0
like_count                0
comment_count             0
description               0
thumbnail_url             0
video_tags                0
kind                      0
language                  0
category_id               0
duration                  0
definition                0
caption                   0
privacy_status            0
license                   0
embeddable                0
live_broadcast_content    0
is_trending               0
engagement_rate           0
collected_at              0
days_since_publish        0
publish_weekday           0
publish_hour              0
is_weekend                0
slot_id                   0
part_of_day               0
category_name             0
views_per_hour            0
is_holiday                0
publish_date_naive  

In [23]:

new_df['category_id'] = new_df['category_id'].fillna(-1)
new_df['category_name'] = new_df['category_name'].fillna("Unknown")


In [ ]:
new_df['desc_len'] = new_df['desc_len'].fillna(0)
new_df['tag_count'] = new_df['tag_count'].fillna(0)


In [ ]:
new_df['description'] = new_df['description'].fillna("No Description")

new_df['video_tags'] = new_df['video_tags'].fillna("No video tags")


In [25]:
# Fill with mode for duration
mode_duration = new_df['duration'].mode()[0]
new_df['duration'] = new_df['duration'].fillna(mode_duration)

# Fill with mode for duration_timedelta
mode_duration_timedelta = new_df['duration_timedelta'].mode()[0]
new_df['duration_timedelta'] = new_df['duration_timedelta'].fillna(mode_duration_timedelta)

# Fill with mode for duration_minutes
mode_duration_minutes = new_df['duration_minutes'].mode()[0]
new_df['duration_minutes'] = new_df['duration_minutes'].fillna(mode_duration_minutes)


In [35]:
new_df.isnull().sum()   

title                     0
channel_name              0
video_id                  0
channel_id                0
publish_date              0
snapshot_date             0
country                   0
view_count                0
like_count                0
comment_count             0
description               0
thumbnail_url             0
video_tags                0
kind                      0
language                  0
category_id               0
duration                  0
definition                0
caption                   0
privacy_status            0
license                   0
embeddable                0
live_broadcast_content    0
is_trending               0
engagement_rate           0
collected_at              0
days_since_publish        0
publish_weekday           0
publish_hour              0
is_weekend                0
slot_id                   0
part_of_day               0
category_name             0
views_per_hour            0
is_holiday                0
publish_date_naive  

In [38]:
existing_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'title', 'channel_name', 'video_id',
       'channel_id', 'publish_date', 'snapshot_date', 'country', 'view_count',
       'like_count', 'comment_count', 'description', 'thumbnail_url',
       'video_tags', 'kind', 'language', 'category_id', 'duration',
       'days_since_publish', 'publish_weekday', 'publish_hour', 'is_weekend',
       'slot_id', 'part_of_day', 'views_per_hour', 'is_holiday',
       'publish_date_naive', 'title_len', 'desc_len', 'tag_count',
       'has_hashtag', 'views_per_hour_norm', 'duration_timedelta',
       'duration_minutes', 'category_name'],
      dtype='object')

In [39]:
existing_df

,Unnamed: 0.1,Unnamed: 0,title,channel_name,video_id,channel_id,publish_date,snapshot_date,country,view_count,like_count,comment_count,description,thumbnail_url,video_tags,kind,language,category_id,duration,days_since_publish,publish_weekday,publish_hour,is_weekend,slot_id,part_of_day,views_per_hour,is_holiday,publish_date_naive,title_len,desc_len,tag_count,has_hashtag,views_per_hour_norm,duration_timedelta,duration_minutes,category_name
0,0,0,Unblock Me 🚫 | A Tamil Short Film | Jasmine Pa...,Ciel Media,OVeuWEXkwB4,UC82WiLf2OKR9fBk2pSqHkMg,2025-09-04 13:55:10+00:00,2025-09-09 00:00:00+00:00,LK,20090,341,18,The relationship started off unexpectedly but ...,https://i.ytimg.com/vi/OVeuWEXkwB4/maxresdefau...,No video tags,youtube#video,en,24.0,PT14M32S,5.0,3,13,0,85,Afternoon(12-18),139.513889,False,2025-09-04 00:00:00,77,1163,1,0,0.000441,0 days 00:14:32,14.533333,Entertainment
1,1,1,Blocked 😧 | A PEEculiar Story | FT. Arun Karth...,Hotspot,rQ-8ldgpOro,UCcoTqhYZ3uCwRNQfqETyNSQ,2025-09-05 12:30:06+00:00,2025-09-09 00:00:00+00:00,LK,169962,7795,83,#arunkarthi #thaafiaabu #hotspotvideos #hotspo...,https://i.ytimg.com/vi/rQ-8ldgpOro/maxresdefau...,"hotspot,hotspot yt channel,hotspot youtube cha...",youtube#video,en,22.0,PT15M50S,4.0,4,12,0,108,Morning(6-12),1416.350000,True,2025-09-05 00:00:00,71,798,3,0,0.004477,0 days 00:15:50,15.833333,People & Blogs
2,2,2,90’s Husband Weds 2K Wife 👩🏻‍❤️‍👨🏻 - Episode ...,Finally,0XGWtz69RRw,UCi3o8sgPl4-Yt501ShuiEgA,2025-09-04 13:51:18+00:00,2025-09-09 00:00:00+00:00,LK,182189,8158,189,Buy Diwali crackers at 90% discount price from...,https://i.ytimg.com/vi/0XGWtz69RRw/maxresdefau...,"finally,finally videos,90s husband weds 2k wif...",youtube#video,en,23.0,PT19M37S,5.0,3,13,0,85,Afternoon(12-18),1265.201389,False,2025-09-04 00:00:00,98,2732,29,0,0.003999,0 days 00:19:37,19.616667,Comedy
3,3,3,Mannipaayaa - Episode 3 | Ravi vj | Thaafia ab...,CHECKMATE,C5TJ4QxWqDA,UCjuCFCBrOJHEdy4y7kdnHtA,2025-09-05 13:30:07+00:00,2025-09-09 00:00:00+00:00,LK,220415,10174,286,Mannipaayaa - Episode 3 | Ravi vj | Thaafia ab...,https://i.ytimg.com/vi/C5TJ4QxWqDA/maxresdefau...,"Checkmate tamil web series,checkmate series ta...",youtube#video,en,24.0,PT15M52S,4.0,4,13,0,109,Afternoon(12-18),1836.791667,True,2025-09-05 00:00:00,82,1149,22,0,0.005806,0 days 00:15:52,15.866667,Entertainment
4,4,4,காசு இல்லனா மனுஷனா கூடமதிக்கமாட்டாங்க! 😭 | Tam...,Tamil Short Films,JGgMyDKK--E,UC0ox8WV7_uqPW80rfbPj7nQ,2025-09-06 04:30:11+00:00,2025-09-09 00:00:00+00:00,LK,646310,19288,90,#RajaRani #Tamilshortfilms #VeyilonEntertainme...,https://i.ytimg.com/vi/JGgMyDKK--E/maxresdefau...,"veyilon entertainment,tamil short films,tamil ...",youtube#video,en,24.0,PT2M49S,3.0,5,4,1,124,Night(0-6),6732.395833,False,2025-09-06 00:00:00,76,146,25,0,0.021282,0 days 00:02:49,2.816667,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86151,86151,86151,COOL DIY SCHOOL HACKS by 123 GO! SHORTS #shorts,123 GO! SHORTS,MwSQlZQ4npI,UCPvupFhJsfJZAx7OQdx2gBw,2023-10-10 14:15:01+00:00,2023-10-26 00:00:00+00:00,LK,8018479,218064,152,Subscribe to our 123 Go! Shorts 👉https://www.y...,https://i.ytimg.com/vi/MwSQlZQ4npI/mqdefault.jpg,"funny, fun, friends, friend, diy, 123GO!, 123 ...",youtube#video,en-US,26.0,PT1M,16.0,1,14,0,38,Afternoon(12-18),19653.134804,False,2023-10-10,47,1102,46,1,0.016862,0 days 00:01:00,1.000000,Howto & Style
86152,86152,86152,LEO - Official Trailer | Thalapathy Vijay | Lo...,Sun TV,Po3jStA673E,UCBnxEdpoZwstJqC1yZpOjRA,2023-10-05 13:00:12+00:00,2023-10-26 00:00:00+00:00,LK,59491216,3115782,166376,Presenting the most anticipated official trail...,https://i.ytimg.com/vi/Po3jStA673E/mqdefault.jpg,"Leo Official trailer, Leo Trailer, Leo movie t...",youtube#video,ta,24.0,PT2M42S,21.0,3,13,0,85,Afternoon(12-18),112672.757576,False,2023-10-05,82,1143,26,0,0.096671,0 days 00:02:42,2.700000,Entertainment
86153,86153,86153,Deweni Inima (දෙවෙන

In [37]:
for each in new_df.columns:
    if each not in existing_df.columns:
        print(each)

definition
caption
privacy_status
license
embeddable
live_broadcast_content
is_trending
engagement_rate
collected_at


In [41]:
cols = [ 'title', 'channel_name', 'video_id',
       'channel_id', 'publish_date', 'snapshot_date', 'country', 'view_count',
       'like_count', 'comment_count', 'description', 'thumbnail_url',
       'video_tags', 'kind', 'language', 'category_id', 'duration',
       'days_since_publish', 'publish_weekday', 'publish_hour', 'is_weekend',
       'slot_id', 'part_of_day', 'views_per_hour', 'is_holiday',
       'publish_date_naive', 'title_len', 'desc_len', 'tag_count',
       'has_hashtag', 'views_per_hour_norm', 'duration_timedelta',
       'duration_minutes', 'category_name']
df_to_integrate_new = new_df.loc[:, cols].copy()

In [46]:
cols = [ 'title', 'channel_name', 'video_id',
       'channel_id', 'publish_date', 'snapshot_date', 'country', 'view_count',
       'like_count', 'comment_count', 'description', 'thumbnail_url',
       'video_tags', 'kind', 'language', 'category_id', 'duration',
       'days_since_publish', 'publish_weekday', 'publish_hour', 'is_weekend',
       'slot_id', 'part_of_day', 'views_per_hour', 'is_holiday',
       'publish_date_naive', 'title_len', 'desc_len', 'tag_count',
       'has_hashtag', 'views_per_hour_norm', 'duration_timedelta',
       'duration_minutes', 'category_name']
df_to_integrate = existing_df.loc[:, cols].copy()

In [47]:
df_final = pd.concat([df_to_integrate_new, df_to_integrate], ignore_index=True)
df_final.dtypes

title                   object
channel_name            object
video_id                object
channel_id              object
publish_date            object
snapshot_date           object
country                 object
view_count               int64
like_count               int64
comment_count            int64
description             object
thumbnail_url           object
video_tags              object
kind                    object
language                object
category_id            float64
duration                object
days_since_publish     float64
publish_weekday          int64
publish_hour             int64
is_weekend               int64
slot_id                  int64
part_of_day             object
views_per_hour         float64
is_holiday                bool
publish_date_naive      object
title_len                int64
desc_len               float64
tag_count                int64
has_hashtag              int64
views_per_hour_norm    float64
duration_timedelta      object
duration

In [48]:
df_final.to_csv('FinalProcessedDataset_03.csv', index=False)

In [49]:
df = pd.read_csv('FinalProcessedDataset_03.csv')
df.isnull().sum()

title                  0
channel_name           0
video_id               0
channel_id             0
publish_date           0
snapshot_date          0
country                0
view_count             0
like_count             0
comment_count          0
description            0
thumbnail_url          0
video_tags             0
kind                   0
language               0
category_id            0
duration               0
days_since_publish     0
publish_weekday        0
publish_hour           0
is_weekend             0
slot_id                0
part_of_day            0
views_per_hour         0
is_holiday             0
publish_date_naive     0
title_len              0
desc_len               0
tag_count              0
has_hashtag            0
views_per_hour_norm    0
duration_timedelta     0
duration_minutes       0
category_name          0
dtype: int64

In [50]:
df

,title,channel_name,video_id,channel_id,publish_date,snapshot_date,country,view_count,like_count,comment_count,description,thumbnail_url,video_tags,kind,language,category_id,duration,days_since_publish,publish_weekday,publish_hour,is_weekend,slot_id,part_of_day,views_per_hour,is_holiday,publish_date_naive,title_len,desc_len,tag_count,has_hashtag,views_per_hour_norm,duration_timedelta,duration_minutes,category_name
0,Watch full short film!!👆@kaidoleeroberts429 #s...,Sofi Manassyan,o8rP66qVN-4,UCZFBnnCCO65xMXOdtFz8CfA,2025-09-06 18:14:11+00:00,2025-09-11 00:00:00+00:00,LK,286283,12778,277,No Description,https://i.ytimg.com/vi/o8rP66qVN-4/maxresdefau...,No video tags,youtube#video,en,1.0,PT1M48S,7.0,5,18,1,138,Afternoon(12-18),1491.057292,False,2025-09-06 00:00:00,77,0.0,1,1,0.009381,0 days 00:01:48,1.800000,Film & Animation
1,ToRung short film: 🙏 let's love each other ❤️,ToRung,07TTa5FHqz8,UCXbYlU08sOTBktOtjVsvR6w,2025-09-06 00:41:33+00:00,2025-09-11 00:00:00+00:00,LK,537527,6174,12,❤️Thank you everyone for always supporting the...,https://i.ytimg.com/vi/07TTa5FHqz8/maxresdefau...,"ToRung,To Rung,Phim Hài,Phim Hai,comedy,laushs...",youtube#video,en,23.0,PT37S,7.0,5,0,1,120,Night(0-6),2799.619792,False,2025-09-06 00:00:00,45,139.0,13,0,0.017613,0 days 00:00:37,0.616667,Comedy
2,He messed with a wrong person #movieexplainedi...,CineClarity,6VfcnEBZ2dg,UCOHMJ8XnZL5TKZbY7Pj1yLA,2025-09-07 03:16:44+00:00,2025-09-11 00:00:00+00:00,LK,2693713,0,285,He messed with a wrong person #movieexplainedi...,https://i.ytimg.com/vi/6VfcnEBZ2dg/maxresdefau...,"Movie explained in hindi,movie explanation in ...",youtube#video,en,24.0,PT2M58S,7.0,6,3,1,147,Night(0-6),14029.755208,True,2025-09-07 00:00:00,60,396.0,17,1,0.088266,0 days 00:02:58,2.966667,Entertainment
3,সব কুলি চোর না। short film _natok3.2_bsngla na...,Natok3.2,BwR2GU0kovc,UCKkv7Lm0fprBm5LSaaKlZoA,2025-09-06 17:26:54+00:00,2025-09-11 00:00:00+00:00,LK,72606,723,1,#foryou \n#natok\n#banglamedia2.3\n#bsngla nat...,https://i.ytimg.com/vi/BwR2GU0kovc/maxresdefau...,No video tags,youtube#video,en,24.0,PT3M1S,7.0,5,17,1,137,Afternoon(12-18),378.156250,False,2025-09-06 00:00:00,50,1591.0,1,0,0.002379,0 days 00:03:01,3.016667,Entertainment
4,😱Movie Recap Short Film Mystery Explained in H...,HowTo1C,EYBCNN_9E2M,UCtkFE68zgVgQimnkgJBSVmA,2025-09-07 09:30:08+00:00,2025-09-11 00:00:00+00:00,LK,22894,0,1,😱Movie Recap Short Film Mystery Explained in H...,https://i.ytimg.com/vi/EYBCNN_9E2M/maxresdefau...,No video tags,youtube#video,en,24.0,PT37S,7.0,6,9,1,153,Morning(6-12),119.239583,True,2025-09-07 00:00:00,98,2709.0,1,1,0.000750,0 days 00:00:37,0.616667,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97761,COOL DIY SCHOOL HACKS by 123 GO! SHORTS #shorts,123 GO! SHORTS,MwSQlZQ4npI,UCPvupFhJsfJZAx7OQdx2gBw,2023-10-10 14:15:01+00:00,2023-10-26 00:00:00+00:00,LK,8018479,218064,152,Subscribe to our 123 Go! Shorts 👉https://www.y...,https://i.ytimg.com/vi/MwSQlZQ4npI/mqdefault.jpg,"funny, fun, friends, friend, diy, 123GO!, 123 ...",youtube#video,en-US,26.0,PT1M,16.0,1,14,0,38,Afternoon(12-18),19653.134804,False,2023-10-10,47,1102.0,46,1,0.016862,0 days 00:01:00,1.000000,Howto & Style
97762,LEO - Official Trailer | Thalapathy Vijay | Lo...,Sun TV,Po3jStA673E,UCBnxEdpoZwstJqC1yZpOjRA,2023-10-05 13:00:12+00:00,2023-10-26 00:00:00+00:00,LK,59491216,3115782,166376,Presenting the most anticipated official trail...,https://i.ytimg.com/vi/Po3jStA673E/mqdefault.jpg,"Leo Official trailer, Leo Trailer, Leo movie t...",youtube#video,ta,24.0,PT2M42S,21.0,3,13,0,85,Afternoon(12-18),112672.757576,False,2023-10-05,82,1143.0,26,0,0.096671,0 days 00:02:42,2.700000,Entertainment
97763,Deweni Inima (දෙවෙනි ඉනිම) | Season 02 | Episo...,TV Derana,6Ja3CbxIh7Y,UCRDDfbYPHX_GUJ4lcQYTc8A,2023-10-24 15:30:08+00:00,2023-10-26 00:00:00+00:00,LK,576815,18697,725,Deweni Inima (දෙවෙනි ඉනිම) | Season 02 | Episo...,https://i.ytimg.com/vi/